# L3: Jamba Prompting and Documents

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import libraries

In [ ]:
from ai21 import AI21Client
from ai21.models.chat import ChatMessage, DocumentSchema
import json

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

## Load API key and create AI21Client

In [ ]:
from utils import get_ai21_api_key
ai21_api_key = get_ai21_api_key()

client = AI21Client(api_key=ai21_api_key)

In [ ]:
messages = [ChatMessage(role='system', content='You are a concise factual based question answering assistant.'),
            ChatMessage(role='user', content='Explain SEC filings in one sentence.')
            ]

In [ ]:
response = client.chat.completions.create(messages=messages,
                                          model='jamba-1.5-large',
                                          # max_tokens=4096,
                                          # temperature=0.4,
                                          # top_p=1.0,
                                          # stop = [], ## ['####', '\n'],
                                          # n=1,
                                          # stream = False
                                          )

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

In [ ]:
response

In [ ]:
response.choices[0].message.content

## JSON output

In [ ]:
messages = [ChatMessage(role='system', 
                        content='You are an assistant that provides information in JSON format.'),
            ChatMessage(role='user', 
                        content=('List top 5 most common type of SEC '
                                 'filings in JSON format, including '
                                 'only form name and description'))]

In [ ]:
response = client.chat.completions.create(
    messages=messages,
    model="jamba-1.5-large",
    temperature = 0,
    response_format={"type": "json_object"}
)

In [ ]:
print(json.dumps(json.loads(response.choices[0].message.content), indent=2))

## Documents object

In [ ]:
## The NVIDIA 10K filing data used in this lesson can be found online here 
## https://www.nasdaq.com/market-activity/stocks/nvda/sec-filings
Filing2023_text = open('Nvidia_10K_20230129.txt', 'r', encoding='utf-8').read()
Filing2024_text = open('Nvidia_10K_20240128.txt', 'r', encoding='utf-8').read()

In [ ]:
Filing2023_doc = DocumentSchema(
    content=Filing2023_text,
    metadata={'company': 'Nvida', 'doc_type': 'SEC annual financial earnings report', 'year': '2023'}
)

Filing2024_doc = DocumentSchema(
    content=Filing2024_text,
    metadata={'company': 'Nvida', 'doc_type': 'SEC annual financial earnings report', 'year': '2024'}
)

documents = [Filing2023_doc, Filing2024_doc]

In [ ]:
messages = [ChatMessage(
                role='system', 
                content='You are a concise factual based question answering assistant. Your answer must be grounded with the document provided.'),
            ChatMessage(
                role='user', 
                content=('Create an html table of the financial performance '
                         'from both of these separate Nvidia 10K filings. '
                         'Include the fiscal year, revenue, and earnings per share'))]

response = client.chat.completions.create(messages=messages,
                                          model="jamba-1.5-large",
                                          documents=documents
                                          )

print(response.choices[0].message.content)

In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML(response.choices[0].message.content))

In [ ]:
response.usage